In [1]:
%reload_ext tensorboard

In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.datasets.mnist import load_data
data = pd.read_csv("train.csv")

X_col = [col for col in data.columns if col not in ['label']]
(X_train_full, y_train_full), (X_test_full, y_test_full) = load_data()
X_train_full = np.reshape(X_train_full, (-1, 784))
X_test_full = np.reshape(X_test_full, (-1, 784))
X_full = data[X_col]
y_full = data['label']
X_valid, X_train = X_test_full/255.0, X_train_full/255.0
y_valid, y_train = y_test_full, y_train_full

X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [3]:
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

def build_model(n_hidden=5, n_neurons=256):
    model = keras.models.Sequential()
    model.add(Dense(n_neurons, activation='elu', kernel_initializer='he_normal', input_shape=[784]))
    keras.layers.Dropout(rate=0.4)
    for layer in range(n_hidden-1):
        keras.layers.BatchNormalization()
        model.add(Dense(n_neurons, activation='elu', kernel_initializer='he_normal'))
        keras.layers.Dropout(rate=0.5)
    keras.layers.BatchNormalization()
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                 optimizer='Nadam',
                 metrics=['accuracy'])
    return model

model = build_model()

In [4]:
from datetime import datetime 

logdir = "logs/scalars" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [5]:
#early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, 
#                                                 restore_best_weights=True)

checkpoint_cb = keras.callbacks.ModelCheckpoint("keras_mnist.h5",
save_best_only=True)

history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_callback, checkpoint_cb])

Epoch 1/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2379 - accuracy: 0.9278 - val_loss: 0.1401 - val_accuracy: 0.9580
Epoch 2/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1308 - accuracy: 0.9607 - val_loss: 0.1446 - val_accuracy: 0.9578
Epoch 3/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1074 - accuracy: 0.9674 - val_loss: 0.1159 - val_accuracy: 0.9663
Epoch 4/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0880 - accuracy: 0.9740 - val_loss: 0.1042 - val_accuracy: 0.9696
Epoch 5/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0809 - accuracy: 0.9768 - val_loss: 0.1094 - val_accuracy: 0.9711
Epoch 6/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0712 - accuracy: 0.9791 - val_loss: 0.1330 - val_accuracy: 0.9661
Epoch 7/100
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0620 - accuracy: 0.9821 - val_loss: 0.1145 - val_ac

1875/1875 [==============================] - 7s 4ms/step - loss: 0.0158 - accuracy: 0.9973 - val_loss: 0.2463 - val_accuracy: 0.9805
Epoch 58/100
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0113 - accuracy: 0.9978 - val_loss: 0.2515 - val_accuracy: 0.9810
Epoch 59/100
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0232 - accuracy: 0.9962 - val_loss: 0.2200 - val_accuracy: 0.9797
Epoch 60/100
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0114 - accuracy: 0.9977 - val_loss: 0.3345 - val_accuracy: 0.9795
Epoch 61/100
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0232 - accuracy: 0.9960 - val_loss: 0.2056 - val_accuracy: 0.9775
Epoch 62/100
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0152 - accuracy: 0.9975 - val_loss: 0.2038 - val_accuracy: 0.9799
Epoch 63/100
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0175 - accuracy: 0.9975 - val_loss: 0.2552 - val_accuracy

In [6]:
%tensorboard --logdir logs/scalars

In [15]:
model = keras.models.load_model("keras_mnist.h5")

model_new = keras.models.Sequential(model.layers[:-1])
model_new.add(keras.layers.Dense(10, activation='softmax'))

for layer in model_new.layers[:-1]:
    layer.trainable = False
    
model_new.compile(loss='sparse_categorical_crossentropy',
                 optimizer='Nadam',
                 metrics=['accuracy'])

X_Valid, X_Train = X_full[36000:]/255.0, X_full[:36000]/255.0
y_Valid, y_Train = y_full[36000:], y_full[:36000]

In [16]:
history = model_new.fit(X_Train, y_Train, epochs=10,
                    validation_data=(X_Valid, y_Valid))

Epoch 1/10
1125/1125 [==============================] - 2s 1ms/step - loss: 0.3592 - accuracy: 0.9450 - val_loss: 0.0391 - val_accuracy: 0.9903
Epoch 2/10
1125/1125 [==============================] - 1s 1ms/step - loss: 0.0317 - accuracy: 0.9918 - val_loss: 0.0335 - val_accuracy: 0.9915
Epoch 3/10
1125/1125 [==============================] - 1s 1ms/step - loss: 0.0260 - accuracy: 0.9930 - val_loss: 0.0316 - val_accuracy: 0.9925
Epoch 4/10
1125/1125 [==============================] - 1s 1ms/step - loss: 0.0236 - accuracy: 0.9936 - val_loss: 0.0317 - val_accuracy: 0.9928
Epoch 5/10
1125/1125 [==============================] - 2s 1ms/step - loss: 0.0217 - accuracy: 0.9939 - val_loss: 0.0359 - val_accuracy: 0.9913
Epoch 6/10
1125/1125 [==============================] - 1s 1ms/step - loss: 0.0207 - accuracy: 0.9942 - val_loss: 0.0305 - val_accuracy: 0.9928
Epoch 7/10
1125/1125 [==============================] - 1s 1ms/step - loss: 0.0193 - accuracy: 0.9946 - val_loss: 0.0301 - val_accuracy:

In [21]:
for layer in model_new.layers[:-1]:
    layer.trainable = True
    
model_new.compile(loss='sparse_categorical_crossentropy',
                 optimizer='Nadam',
                 metrics=['accuracy'])

In [23]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("keras_mnist_final.h5",
save_best_only=True)

history = model.fit(X_Train, y_Train, epochs=100,
                    validation_data=(X_Valid, y_Valid),
                    callbacks=[ checkpoint_cb])

Epoch 1/100
1125/1125 [==============================] - 4s 4ms/step - loss: 0.0447 - accuracy: 0.9876 - val_loss: 0.0482 - val_accuracy: 0.9893
Epoch 2/100
1125/1125 [==============================] - 4s 4ms/step - loss: 0.0416 - accuracy: 0.9886 - val_loss: 0.0451 - val_accuracy: 0.9863
Epoch 3/100
1125/1125 [==============================] - 4s 4ms/step - loss: 0.0613 - accuracy: 0.9880 - val_loss: 0.0701 - val_accuracy: 0.9842
Epoch 4/100
1125/1125 [==============================] - 5s 4ms/step - loss: 0.0219 - accuracy: 0.9941 - val_loss: 0.0945 - val_accuracy: 0.9825
Epoch 5/100
1125/1125 [==============================] - 4s 4ms/step - loss: 0.0271 - accuracy: 0.9924 - val_loss: 0.1197 - val_accuracy: 0.9788
Epoch 6/100
1125/1125 [==============================] - 4s 4ms/step - loss: 0.0278 - accuracy: 0.9926 - val_loss: 0.0661 - val_accuracy: 0.9897
Epoch 7/100
1125/1125 [==============================] - 4s 4ms/step - loss: 0.0293 - accuracy: 0.9928 - val_loss: 0.1122 - val_ac

1125/1125 [==============================] - 4s 4ms/step - loss: 0.0260 - accuracy: 0.9965 - val_loss: 0.2029 - val_accuracy: 0.9802
Epoch 58/100
1125/1125 [==============================] - 4s 4ms/step - loss: 0.0153 - accuracy: 0.9973 - val_loss: 0.2433 - val_accuracy: 0.9795
Epoch 59/100
1125/1125 [==============================] - 4s 4ms/step - loss: 0.0231 - accuracy: 0.9976 - val_loss: 0.2358 - val_accuracy: 0.9805
Epoch 60/100
1125/1125 [==============================] - 4s 4ms/step - loss: 0.0111 - accuracy: 0.9983 - val_loss: 0.2196 - val_accuracy: 0.9830
Epoch 61/100
1125/1125 [==============================] - 4s 4ms/step - loss: 0.0188 - accuracy: 0.9967 - val_loss: 0.2557 - val_accuracy: 0.9742
Epoch 62/100
1125/1125 [==============================] - 4s 4ms/step - loss: 0.0136 - accuracy: 0.9975 - val_loss: 0.2141 - val_accuracy: 0.9780
Epoch 63/100
1125/1125 [==============================] - 4s 4ms/step - loss: 0.0140 - accuracy: 0.9973 - val_loss: 0.9944 - val_accuracy

In [24]:
model_new = keras.models.load_model("keras_mnist_final.h5")
test = pd.read_csv("test.csv")

In [26]:
y_test = model.predict_classes(test/255)

output = pd.DataFrame({"ImageId": test.index+1, 'Label':y_test})

print(model_new.evaluate(X_full/255, y_full))
print(model.evaluate(X_full/255, y_full))

1313/1313 [==============================] - 2s 2ms/step - loss: 0.0232 - accuracy: 0.9930
[0.023189948871731758, 0.9929524064064026]
1313/1313 [==============================] - 2s 2ms/step - loss: 0.0448 - accuracy: 0.9970
[0.04479839280247688, 0.9969761967658997]


In [27]:
output.to_csv('sub.csv', index=False)